# 126 Project Traffic Simulator

Let the road be length $n$. Cars start at position 0 and reach their destination at position $n-1$. Each car has an exponential "clock" with parameter $\gamma$, and every time their clock goes off they move forward with probability $p$. If a car is directly in front of them, they do not move regardless. A car enters the road at position 0 at a rate of $Exp(\alpha)$ and exit the road at position $n-1$ with a rate of $Exp(\beta)$.

### Choose global variables:

In [ ]:
from ipywidgets import interactive
from IPython.display import display
from IPython.display import clear_output

def set_params(n, alpha, beta, gamma, p, seed):
    return [n, alpha, beta, gamma, p, seed]

params = interactive(set_params, n=(1, 20), alpha=(0.0, 2.0), beta=(0.0, 2.0), gamma=(0.0, 2.0), p=(0.0, 1.0), seed=(1, 999))
display(params)

### Set global variables:

In [ ]:
n, alpha, beta, gamma, p, seed = params.result

### Define particle and environment classes:

In [ ]:
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output


class Particle:
    
    count = 0
    
    def __init__(self, gamma=gamma, p=p):
        Particle.count += 1
        self.id = Particle.count - 1
        self.gamma = gamma
        self.p = p
        self._position = 0
        
    def update_position(self, value=1):
        self._position += value
        
    def get_position(self):
        return self._position
        
class Environment:
    def __init__(self, n=n, alpha=alpha, beta=beta):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.particles = []
        self.occupied_positions = [0 for _ in range(n)]
        
    def run_simulation(self, time_len=60):
        sys.stdout.write("\r{}".format(self.occupied_positions))
        sys.stdout.flush()
        
        curr_time = 0
        time_lapse = np.random.exponential(scale=1/alpha)
        time.sleep(time_lapse)
        curr_time += time_lapse
        particle = Particle()
        self.particles.append(particle)
        self.occupied_positions[0] = 1
        
        actions = [(curr_time, particle.id, particle.get_position())]
        entrances = 1
        exits = 0
        
        while curr_time < time_len:
            sys.stdout.write("\r{}".format(self.occupied_positions))
            sys.stdout.flush()
            
            merged_param = sum([particle.gamma * particle.p for particle in self.particles]) + self.alpha + self.beta
            time_lapse = np.random.exponential(scale=1/merged_param)
            probabilities = np.array([particle.gamma * particle.p for particle in self.particles] + [self.alpha, self.beta]) / merged_param
            particle = np.random.choice(self.particles + ['entrance', 'exit'], p=probabilities)
            time.sleep(time_lapse)
            curr_time += time_lapse
            if particle == 'entrance':
                if self.occupied_positions[0]:
                    continue
                else:
                    new_particle = Particle()
                    self.particles.append(new_particle)
                    self.occupied_positions[0] = 1
                    
                    actions += [(curr_time, new_particle.id, new_particle.get_position())]
                    entrances += 1
                    
            elif particle == 'exit':
                if self.occupied_positions[n-1]:
                    end_particle = self.particles.pop(0)
                    self.occupied_positions[n-1] = 0
                    
                    actions += [(curr_time, end_particle.id, 'exit')]
                    exits += 1
                    
                else:
                    continue
            else:
                particle_pos = particle.get_position()
                if particle_pos == n - 1 or self.occupied_positions[particle_pos + 1] == 1:
                    continue
                else:
                    self.occupied_positions[particle_pos] = 0
                    self.occupied_positions[particle_pos + 1] = 1
                    particle.update_position()
                    
                    actions += [(curr_time, particle.id, particle.get_position())]
                    
        print('\n done')
        print('Summary:')
        print('Total entrances: {}'.format(entrances))
        print('Total exits: {}'.format(exits))
        print('\n Log:')
        for entry in actions:
            print('Time: {}'.format(entry[0]))
            print('Particle: {}'.format(entry[1]))
            print('New position: {}'.format(entry[2]))
            print('\n')

### Add 2D Environment:

In [ ]:
class Particle2D(Particle):
    def __init__(self, gamma=gamma, p=p, direction=0): # 0 is horizontal, 1 is vertical
        Particle.__init__(self, gamma, p)
        self._position = [-1, -1]
        self._direction = direction
        
    def update_position(self, value=1):
        self._position[self._direction] += value
        
    def set_position(self, value=[-1, -1]):
        self._position = value
        
    def get_direction(self):
        return self._direction
        
# FIRST INDEX COLUMN, SECOND INDEX ROAD
class Env2D(Environment):
    def __init__(self, num_roads=(1,1), n=n, alpha=alpha, beta=beta):
        Environment.__init__(self, n, alpha, beta)
        self.occupied_positions = np.array([[0 for _ in range(n)] for _ in range(n)])
        self.horizontal_roads = np.random.choice(range(n), num_roads[0], replace=False)
        self.vertical_roads = np.random.choice(range(n), num_roads[1], replace=False)
        self.num_roads = num_roads
        
    def run_simulation(self, time_len=60):
        print(self.occupied_positions)
        
        curr_time = 0
        time_lapse = np.random.exponential(scale=1/(alpha * sum(self.num_roads)))
        time.sleep(time_lapse)
        curr_time += time_lapse
        particle = Particle2D(direction=np.random.choice([0, 1], p=[self.num_roads[0]/sum(self.num_roads), self.num_roads[1]/sum(self.num_roads)]))
        self.particles.append(particle)
        if not particle.get_direction():
            road = np.random.choice(self.horizontal_roads)
            particle.set_position([0, road])
            self.occupied_positions[0, road] = 1
        else:
            road = np.random.choice(self.vertical_roads)
            particle.set_position([road, 0])
            self.occupied_positions[road, 0] = 1
        
        actions = [(curr_time, particle.id, particle.get_position())]
        entrances = 1
        exits = 0
        
        while curr_time < time_len:
            clear_output(wait=True)
            print(self.occupied_positions)
            
            merged_param = sum([particle.gamma * particle.p for particle in self.particles]) + (self.alpha + self.beta) * sum(self.num_roads)  
            time_lapse = np.random.exponential(scale=1/merged_param)
            probabilities = np.array([particle.gamma * particle.p for particle in self.particles] + [self.alpha for _ in range(sum(self.num_roads))] + [self.beta for _ in range(sum(self.num_roads))]) / merged_param
            particle = np.random.choice(self.particles + ['entrance' for _ in range(sum(self.num_roads))] + ['exit' for _ in range(sum(self.num_roads))], p=probabilities)
            time.sleep(time_lapse)
            curr_time += time_lapse
            if particle == 'entrance':
                direction = np.random.choice([0, 1], p=[len(self.horizontal_roads)/sum(self.num_roads), len(self.vertical_roads)/sum(self.num_roads)])
                if not direction:
                    road = np.random.choice(self.horizontal_roads)
                    if self.occupied_positions[0][road]:
                        continue
                    new_particle = Particle2D(direction=direction)
                    self.particles.append(new_particle)
                    self.occupied_positions[0][road] = 1
                    new_particle.set_position([0, road])
                else:
                    road = np.random.choice(self.vertical_roads)
                    if self.occupied_positions[road][0]:
                        continue
                    new_particle = Particle2D(direction=direction)
                    self.particles.append(new_particle)
                    self.occupied_positions[road][0] = 1
                    new_particle.set_position([road, 0])
                
                    
                actions += [(curr_time, new_particle.id, new_particle.get_position())]
                entrances += 1
                    
            elif particle == 'exit':
                direction = np.random.choice([0, 1], p=[len(self.horizontal_roads)/sum(self.num_roads), len(self.vertical_roads)/sum(self.num_roads)])
                if not direction:
                    road = np.random.choice(self.horizontal_roads)
                    if self.occupied_positions[n-1][road]:
                        particle = [p for p in self.particles if p.get_position() == [n-1, road]][0]
                        self.particles.remove(particle)
                        self.occupied_positions[n-1][road] = 0
                        actions += [(curr_time, particle.id, [n, road])]
                        exits += 1
                else:
                    road = np.random.choice(self.vertical_roads)
                    if self.occupied_positions[road][n-1]:
                        particle = [p for p in self.particles if p.get_position() == [road, n-1]][0]
                        self.particles.remove(particle)
                        self.occupied_positions[road][n-1] = 0
                        actions += [(curr_time, particle.id, [road, n])]
                        exits += 1
            else:
                particle_pos = particle.get_position()
                direction = particle.get_direction()
                if not direction:
                    if particle_pos[0] == n - 1 or self.occupied_positions[particle_pos[0] + 1, particle_pos[1]] == 1:
                        continue
                    else:
                        self.occupied_positions[particle_pos[0]][particle_pos[1]] = 0
                        self.occupied_positions[particle_pos[0] + 1][particle_pos[1]] = 1
                        particle.update_position()
                        actions += [(curr_time, particle.id, particle.get_position())]
                else:
                    if particle_pos[1] == n - 1 or self.occupied_positions[particle_pos[0], particle_pos[1] + 1] == 1:
                        continue
                    else:
                        self.occupied_positions[particle_pos[0]][particle_pos[1]] = 0
                        self.occupied_positions[particle_pos[0]][particle_pos[1] + 1] = 1
                        particle.update_position()
                        actions += [(curr_time, particle.id, particle.get_position())]
                    
        print('\n done')
        print('Summary:')
        print('Total entrances: {}'.format(entrances))
        print('Total exits: {}'.format(exits))
        print('\n Log:')
        print(actions)
        for entry in actions:
            print('Time: {}'.format(entry[0]))
            print('Particle: {}'.format(entry[1]))
            print('New position: {}'.format(entry[2]))
            print('\n')

### Run simulator:

In [ ]:
env = Env2D()
env.run_simulation()